## Tuto to find The sources and Objects

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNRS
- member : DESC, rubin-inkind
- creation date : 2025-05-01
- last update : 2025-05-01

# Note on work done on LSSTComCam Commissioning : https://sitcomtn-149.lsst.io/
# Note Data Product definition Document : https://lse-163.lsst.io/

In [ ]:
import sys
import matplotlib.pyplot as plt
import lsst.afw.display as afwDisplay
import numpy as np
import pandas as pd
from astropy.time import Time
# %matplotlib widget

In [ ]:
import lsst.geom as geom
from lsst.geom import SpherePoint, degrees
from lsst.skymap import PatchInfo, Index2D

In [ ]:
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["axes.labelsize"] = "x-large"
plt.rcParams["axes.titlesize"] = "x-large"
plt.rcParams["xtick.labelsize"] = "x-large"
plt.rcParams["ytick.labelsize"] = "x-large"

In [ ]:
import traceback

In [ ]:
# Define butler
from lsst.daf.butler import Butler

In [ ]:
!eups list lsst_distrib

## RubinTV, Campaigns , quicklook
- RubinTV : https://usdf-rsp.slac.stanford.edu/rubintv/summit-usdf/lsstcam
- https://rubinobs.atlassian.net/wiki/spaces/LSSTCOM/pages/467370016/LSSTCam+Commissioning+Planning
- LSSTCam DM campaign : https://rubinobs.atlassian.net/wiki/spaces/DM/pages/48834013/Campaigns#1.1.2.-LSSTCam-Nightly-Validation-Pipeline
- Check campaign also here  https://rubinobs.atlassian.net/wiki/pages/diffpagesbyversion.action?pageId=48834013&selectedPageVersions=145%2C143
- fov-quicklook : https://usdf-rsp-dev.slac.stanford.edu/fov-quicklook/

## Configuration

### Choose instrument

In [ ]:
# instrument = "LSSTCam"
instrument = "LSSTComCam"

### Choose options

### For LSSTCam : RubinTV, Campaigns , quicklook
- RubinTV : https://usdf-rsp.slac.stanford.edu/rubintv/summit-usdf/lsstcam
- https://rubinobs.atlassian.net/wiki/spaces/LSSTCOM/pages/467370016/LSSTCam+Commissioning+Planning
- LSSTCam DM campaign : https://rubinobs.atlassian.net/wiki/spaces/DM/pages/48834013/Campaigns#1.1.2.-LSSTCam-Nightly-Validation-Pipeline
- Check campaign also here  https://rubinobs.atlassian.net/wiki/pages/diffpagesbyversion.action?pageId=48834013&selectedPageVersions=145%2C143
- fov-quicklook : https://usdf-rsp-dev.slac.stanford.edu/fov-quicklook/

### For LSSTComCam check here : 
- Check here the collection available : https://rubinobs.atlassian.net/wiki/spaces/DM/pages/226656354/LSSTComCam+Intermittent+Cumulative+DRP+Runs

In [ ]:
if instrument == "LSSTCam":
    repo = "/repo/embargo"
    instrument = "LSSTCam"
    collection_validation = instrument + "/runs/nightlyValidation"
    # collection_quicklook   = instrument + '/runs/quickLookTesting'
    collection_validation = os.path.join(collection_validation, "20250416/d_2025_04_15/DM-50157")
    date_start = 20250415
    date_selection = 20250416
    where_clause = "instrument = '" + f"{instrument}" + "'"
    where_clause_date = where_clause + f"and day_obs >= {date_start}"
    skymapName = "lsst_cells_v1"
    BANDSEL = "i"

elif instrument == "LSSTComCam":
    repo = "/repo/main"
    collection_validation = "LSSTComCam/runs/DRP/DP1/w_2025_10/DM-49359"  # work
    date_start = 20241024
    date_selection = 20241211
    skymapName = "lsst_cells_v1"
    where_clause = "instrument = '" + instrument + "'"
    where_clause_date = where_clause + f"and day_obs >= {date_start}"

    NDET = 9
    TRACTSEL = 5063
    BANDSEL = "i"

In [ ]:
collectionStr = collection_validation.replace("/", "_")

## Access to Butler registry

In [ ]:
# Initialize the butler repo:
butler = Butler(repo, collections=collection_validation)
registry = butler.registry

## Create a skymap object and Camera

In [ ]:
skymap = butler.get("skyMap", skymap=skymapName, collections=collection_validation)

In [ ]:
camera = butler.get("camera", collections=collection_validation, instrument=instrument)

## Query for collections in Butler

- remove user collections
- remove calibration products

In [ ]:
# mostly setup for LSSTCam
for _ in sorted(registry.queryCollections(expression=instrument + "/*")):
    if "/calib/" not in _ and "u/" not in _:
        print(_)

## Query for the dataset types in the Butler

- Refer to the Data Product definition Document to know about the definition of datasets
- https://www.lsst.org/about/dm/data-products
- https://lse-163.lsst.io/
- https://docushare.lsst.org/docushare/dsweb/Get/LSE-163

In [ ]:
for datasetType in registry.queryDatasetTypes():
    if registry.queryDatasets(datasetType, collections=collection_validation).any(execute=False, exact=False):
        # Limit search results to the data products
        if (
            ("_config" not in datasetType.name)
            and ("_log" not in datasetType.name)
            and ("_metadata" not in datasetType.name)
            and ("_resource_usage" not in datasetType.name)
            and ("Plot" not in datasetType.name)
            and ("Metric" not in datasetType.name)
            and ("metric" not in datasetType.name)
        ):
            if "object" in datasetType.name or "Obj" in datasetType.name:
                print(datasetType)
            if "source" in datasetType.name or "Source" in datasetType.name:
                print(datasetType)

## Region of interest

In [ ]:
lsstcomcam_targets = {}
lsstcomcam_targets["47 Tuc"] = {"field_name": "47 Tuc Globular Cluster", "ra": 6.02, "dec": -72.08}
lsstcomcam_targets["Rubin SV 38 7"] = {"field_name": "Low Ecliptic Latitude Field", "ra": 37.86, "dec": 6.98}
lsstcomcam_targets["Fornax dSph"] = {
    "field_name": "Fornax Dwarf Spheroidal Galaxy",
    "ra": 40.0,
    "dec": -34.45,
}
lsstcomcam_targets["ECDFS"] = {"field_name": "Extended Chandra Deep Field South", "ra": 53.13, "dec": -28.10}
lsstcomcam_targets["EDFS"] = {"field_name": "Euclid Deep Field South", "ra": 59.10, "dec": -48.73}
lsstcomcam_targets["Rubin SV 95 -25"] = {
    "field_name": "Low Galactic Latitude Field",
    "ra": 95.00,
    "dec": -25.0,
}
lsstcomcam_targets["Seagull"] = {"field_name": "Seagull Nebula", "ra": 106.23, "dec": -10.51}

In [ ]:
# the_target = lsstcomcam_targets["Seagull"]
# the_target = lsstcomcam_targets["47 Tuc"] # bad
# the_target = lsstcomcam_targets["Fornax dSph"]
# the_target = lsstcomcam_targets["ECDFS"]


# key = "Seagull"
# key = "Fornax dSph"
key = "ECDFS"
# key = "EDFS"
# key = "47 Tuc"
# key = "Rubin SV 38 7"
# key = "Rubin SV 95 -25"


the_target = lsstcomcam_targets[key]
target_ra = the_target["ra"]
target_dec = the_target["dec"]
target_title = (
    the_target["field_name"] + f" band  {BANDSEL} " + f" (ra,dec) = ({target_ra:.2f},{target_dec:.2f}) "
)
target_point = SpherePoint(target_ra, target_dec, degrees)

## Get list of tracts from the objectTable_tract

In [ ]:
# Find the dimension
print(butler.registry.getDatasetType("objectTable_tract").dimensions)

In [ ]:
datasettype = "objectTable_tract"
therefs = butler.registry.queryDatasets(datasettype, collections=collection_validation)
tractsId_list = np.unique([ref.dataId["tract"] for ref in therefs])
tractsId_list = sorted(tractsId_list)
print(tractsId_list)

## Find the Tract and Patch of the region of interest

- tract in tractNbSel
- patch in patchNbSel

In [ ]:
tract_info = skymap.findTract(target_point)
patch_info = tract_info.findPatch(target_point)
bbox = patch_info.getOuterBBox()

print("Patch bounding box:", bbox)

print("Tract ID :", tract_info.getId())
tractNbSel = tract_info.getId()

print("Patch Index :", patch_info.getIndex(), " , ", patch_info.getSequentialIndex())  # (x, y)
print("Bounding Box", bbox)

patchNbSel = patch_info.getSequentialIndex()

## The Objects

In [ ]:
print(butler.registry.getDatasetType("objectTable").dimensions)

In [ ]:
where_clause = f"skymap = '{skymapName}' AND tract = {tractNbSel} AND patch = {patchNbSel}"
print(where_clause)

In [ ]:
dataset_refs = list(
    butler.registry.queryDatasets("objectTable", collections=collection_validation, where=where_clause)
)

# Récupère un des refs valides
ref = dataset_refs[0]

# Utilise ref.datasetType.name pour lister les colonnes disponibles
t_columns = list(butler.get(ref).columns)
print(t_columns)

Oui, dans ce type de objectTable, les colonnes comme g_psfFlux, g_kronFlux, g_cModelFlux, etc., sont des flux calibrés (en nJy ou en unité de calibration interne du pipeline). Pour les convertir en magnitudes AB, tu peux utiliser la formule classique :

In [ ]:
# La constante 31.4 correspond à la conversion AB standard pour un flux exprimé en nanoJanskys (nJy).
# À vérifier selon l’unité exacte utilisée par le pipeline sur ton RSP
# (souvent c’est bien nJy, mais ça peut être autre chose si la calibration a été changée).
# mag = -2.5 * np.log10(flux) + 31.4